In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot
import seaborn as sns
import tensorflow as tf
import IPython
import IPython.display
from statsmodels.tsa.vector_ar.var_model import VAR
import sklearn
from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.vector_ar.var_model import VAR

# fix for 'package not found' when installing in Anaconda environment
if 'google.colab' not in str(get_ipython()):
    import pip
    pip.main(['install', 'xgboost'])

if 'google.colab' in str(get_ipython()):
  !rm util.py
  !rm window.py
  !rm models.py
  !wget https://raw.githubusercontent.com/marco-mazzoli/progetto-tesi/master/util.py
  !wget https://raw.githubusercontent.com/marco-mazzoli/progetto-tesi/master/windows.py
  !wget https://raw.githubusercontent.com/marco-mazzoli/progetto-tesi/master/models.py

from xgboost import XGBRegressor
from util import select_relevant_rows, select_attributes, read_movement_data,download_updated_mobility_data, download_updated_mobility_data, series_to_supervised
from windows import WindowGenerator
from models import MultiStepLastBaseline

# Data Acquisition and Data Preparation

In [ ]:
local_region_path = r'../COVID-19/dati-regioni/dpc-covid19-ita-regioni.csv'
remote_region_path = r'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

regions_frame = pd.read_csv(remote_region_path)

In [ ]:
region_focus = 'Emilia-Romagna'
attribute_focus = 'denominazione_regione'

region_focus_data = select_relevant_rows(
    regions_frame,
    attribute_focus,
    region_focus
    )

In [ ]:
frame_interesting_columns = select_attributes(region_focus_data, [
    'data',
    'ricoverati_con_sintomi',
    'terapia_intensiva',
    'totale_ospedalizzati',
    'variazione_totale_positivi',
    'nuovi_positivi',
    'deceduti',
    'tamponi',
    'ingressi_terapia_intensiva'
    ])

In [ ]:
frame_interesting_columns.tail()

In [ ]:
frame_interesting_columns = pd.DataFrame(frame_interesting_columns)
frame_interesting_columns['data'] = pd.to_datetime(frame_interesting_columns['data'])
frame_interesting_columns['data'] = frame_interesting_columns['data'].dt.strftime(r'%Y-%m-%d')
frame_interesting_columns = frame_interesting_columns.fillna(0)


In [ ]:
mobility_data_url = r'https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv'
file_path = r'../Global_Mobility_Report.csv'
mobility_data_zip_url = r'https://www.gstatic.com/covid19/mobility/Region_Mobility_Report_CSVs.zip'
zip_path = r'../Region_Mobility_Report_CSVs.zip'
region_mobility_path = r'../Region_Mobility_Report_CSVs'

download_updated_mobility_data(
    mobility_data_url,
    file_path,
    region_mobility_path,
    mobility_data_zip_url,
    zip_path
    )

mobility_df = read_movement_data(
    region_mobility_path,
    'IT_Region_Mobility_Report',
    region='Emilia-Romagna'
    )

In [ ]:
mobility_df = mobility_df[['date',
            'retail_and_recreation_percent_change_from_baseline',
            'grocery_and_pharmacy_percent_change_from_baseline',
            'parks_percent_change_from_baseline',
            'transit_stations_percent_change_from_baseline',
            'workplaces_percent_change_from_baseline',
            'residential_percent_change_from_baseline']].fillna(0)

In [ ]:
frame_interesting_columns.rename(columns={'data': 'date'}, inplace=True)
frame_interesting_columns.set_index('date',inplace=True)
mobility_df.set_index('date',inplace=True)
merged = pd.merge(
    frame_interesting_columns,
    mobility_df,
    on='date'
    )
merged = merged.fillna(0)
merged.set_index(pd.DatetimeIndex(merged.index))

In [ ]:
fig, axes = pyplot.subplots(nrows=4,ncols=2,figsize=(10,8))
for i, ax in enumerate(axes.flatten()):
    data = np.array(frame_interesting_columns[frame_interesting_columns.columns[i]])
    ax.plot(pd.DataFrame(data))
    ax.set_title(frame_interesting_columns.columns[i])
    ax.plot()

pyplot.tight_layout()


In [ ]:
fig, axes = pyplot.subplots(nrows=3,ncols=2,figsize=(10,8))
for i, ax in enumerate(axes.flatten()):
    data = np.array(mobility_df[mobility_df.columns[i]])
    ax.plot(pd.DataFrame(data))
    ax.set_title(mobility_df.columns[i])
    ax.plot()

pyplot.tight_layout()


In [ ]:
df = merged
n = len(merged)
train_df = df[0:int(n*0.7)]
val_df = df[int(n*0.7):int(n*0.9)]
test_df = df[int(n*0.9):]

num_features = df.shape[1]
num_features

# Univariate XGBOOST

In [ ]:
column_univariate = merged['nuovi_positivi']

reframed = series_to_supervised(pd.DataFrame(column_univariate),window=7)

In [ ]:
reframed

In [ ]:
def train_test_split(data,n_test):
	return data[:-n_test, :], data[-n_test:, :]

def walk_forward_validation(data,n_test):
    predictions = list()
    train, test = train_test_split(data,n_test)
    history = [x for x in train]
    for i in range(len(test)):
        test_X, test_y = test[i, :-1], test[i, -1]
        prediction = xgboost_forecast(history,test_X)
        predictions.append(prediction)
        history.append(test[i])
    error = mean_absolute_error(test[:, -1],predictions)
    return error, test[:, 1], predictions

def xgboost_forecast(train,test_X):
    train = np.ascontiguousarray(train)
    test_X = np.ascontiguousarray(test_X)
    train_X, train_y = train[:, :-1], train[:, -1]
    model = XGBRegressor(objective='reg:squarederror',n_estimators=1000)
    model.fit(train_X, train_y)
    prediction = model.predict([test_X])
    return prediction[0]

In [ ]:
mae, y, yhat = walk_forward_validation(reframed.values,7)

In [ ]:
print('MAE: %.3f' % mae)
pyplot.plot(y,label='Expected')
pyplot.plot(yhat,label='Predicted')
pyplot.legend()
pyplot.show()

# VAR (Vector Auto Regression)

In [ ]:
# !pip3 install scikit-learn==0.24

In [ ]:
# n_splits=10
# test_size=5

# tscv = TimeSeriesSplit(n_splits=n_splits,test_size=test_size)

# df = frame_interesting_columns

# for train_index, test_index in tscv.split(df):
#     train = df.iloc[train_index]
#     test = df.iloc[test_index]

#     model = VAR(endog=train,freq='D')
#     fit = model.fit()
#     lag_order = fit.k_ar

#     prediction = fit.forecast(train.values[-lag_order:],len(test_index))

#     pred_df = pd.DataFrame(prediction,columns=df.columns)
 
#     print('train interval: ' + str(train.index[0]) + ' - ' + str(train.index[-1]))
#     print('test interval: ' + str(test.index[0]) + ' - ' + str(test.index[-1]))
#     for i in test.columns:
#         print('mae value for', i, 'is : ', mean_absolute_error(pred_df[i], test[i]))


# LSTM Univariate


In [ ]:
column_univariate = 'nuovi_positivi'
df = pd.DataFrame(frame_interesting_columns[column_univariate])
scaler = MinMaxScaler(feature_range=(-1,1))
scaled = scaler.fit_transform(df)

In [ ]:
reframed = series_to_supervised(pd.DataFrame(scaled),14,1)
values = reframed.values
train = values[:500,:]
test = values[500:,:]

train_X, train_y = train[:,:-1], train[:,-1]
test_X, test_y = test[:,:-1], test[:,-1]

train_X = train_X.reshape((train_X.shape[0],train_X.shape[1],1))
test_X = test_X.reshape((test_X.shape[0],test_X.shape[1],1))

print(train_X.shape,train_y.shape,test_X.shape,test_y.shape)

In [ ]:
reframed.tail()

In [ ]:
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1],train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae',optimizer='adam')

history = model.fit(train_X,train_y,epochs=50,batch_size=72,validation_data=(test_X,test_y),verbose=0,shuffle=False)

pyplot.plot(history.history['loss'],label='Training loss')
pyplot.plot(history.history['val_loss'],label='Validation loss')
pyplot.legend()
pyplot.show()

In [ ]:
train_prediction = model.predict(train_X)
test_prediction = model.predict(test_X)

train_prediction = scaler.inverse_transform(train_prediction)
test_prediction = scaler.inverse_transform(test_prediction)
test_y = scaler.inverse_transform([test_y])


In [ ]:
test_y_descaled = test_y[0]
print('Test error')
print(mean_absolute_error(test_y_descaled,test_prediction))
pyplot.plot(pd.DataFrame(test_y_descaled))
pyplot.plot(pd.DataFrame(test_prediction))
pyplot.show()

In [ ]:
from keras.preprocessing.sequence import TimeseriesGenerator
df = pd.DataFrame(frame_interesting_columns['nuovi_positivi'])

values = df['nuovi_positivi'].values
values = values.reshape((-1,1))

values

split_percent = 0.80
split = int(split_percent*len(values))

train = values[:split]
test = values[split:]

date_train = df.index[:split]
date_test = df.index[split:]

look_back = 7

train_generator = TimeseriesGenerator(train,train,length=look_back,batch_size=20)    
test_generator = TimeseriesGenerator(test,test,length=look_back,batch_size=1)

model = Sequential()
model.add(LSTM(50,activation='relu',input_shape=(look_back,1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mae')

num_epochs = 50
model.fit(train_generator,epochs=num_epochs,verbose=0)

In [ ]:
import plotly.graph_objects as go

prediction = model.predict(test_generator)

train = train.reshape((-1))
test = test.reshape((-1))
prediction = prediction.reshape((-1))


trace1 = go.Scatter(
    x = date_train,
    y = train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "nuovi_positivi",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)

fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

print('mae:')
print(mean_absolute_error(test,prediction))

len(test_generator.targets)
len(prediction)

# Multivariate LSTM

In [ ]:
df = pd.DataFrame(frame_interesting_columns.drop('tamponi',axis=1))
scaler = MinMaxScaler(feature_range=(0,1))
scaled = scaler.fit_transform(df)

In [ ]:
train_X = []
train_y = []

n_future = 1
n_past = 14

dates = df.index

df = scaled

for i in range(n_past,len(df) - n_future + 1):
    train_X.append(df[i - n_past:i, 0:df.shape[1]])
    train_y.append(df[i + n_future - 1:i + n_future, 
    0])

train_X, train_y = np.array(train_X), np.array(train_y)

print('train_X shape == {}.'.format(train_X.shape))
print('train_y shape == {}.'.format(train_y.shape))

model = Sequential()
model.add(LSTM(64,activation='relu',input_shape=(train_X.shape[1],train_X.shape[2]),return_sequences=True))
model.add(LSTM(32,activation='relu',return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(train_y.shape[1]))

model.compile(optimizer='adam',loss='mse')
model.summary()

In [ ]:
history = model.fit(train_X,train_y,epochs=20,batch_size=16,validation_split=0.1,verbose=0)


In [ ]:
pyplot.plot(history.history['loss'],label='Training loss')
pyplot.plot(history.history['val_loss'],label='Validation loss')
pyplot.legend()

In [ ]:
n_lags = 21
n_predictions = 15

In [ ]:
prediction = model.predict(train_X[-n_predictions:])

In [ ]:
prediction_copies = np.repeat(prediction,df.shape[1],axis=-1)
y_pred_future = scaler.inverse_transform(prediction_copies)[:,0]

In [ ]:
predict_period_dates = pd.date_range(dates[-n_past],periods=n_predictions).tolist()
print(predict_period_dates)

In [ ]:
forecast_dates = []
for time_i in predict_period_dates:
    forecast_dates.append(time_i)

In [ ]:
df_forecast = pd.DataFrame({'Date':np.array(forecast_dates),'Feature':y_pred_future})
df_forecast['Date'] = pd.to_datetime(df_forecast['Date'])

In [ ]:
# sns.lineplot(merged['nuovi_positivi'])

# sns.lineplot(df_forecast['Feature'])
# merged['nuovi_positivi']
df_forecast.index = df_forecast['Date']

In [ ]:
df_forecast =  df_forecast.drop('Date',1)

In [ ]:
# sns.lineplot(df_forecast['Feature'])
# sns.lineplot(merged['nuovi_positivi'])
df_forecast['Feature']
df_real = frame_interesting_columns['nuovi_positivi']
df_real = pd.DataFrame(df_real[-n_predictions:])

In [ ]:
df_real

In [ ]:
df_forecast